In [ ]:
#1. Data collection

import pandas as pd
import numpy as np
d=pd.read_csv('/content/drive/MyDrive/ehealthdatset/dataset.txt')
d


,label,text
0,__label__suicidal,my life is meaningless i just want to end my l...
1,__label__suicidal,muttering i wanna die to myself daily for a fe...
2,__label__suicidal,work slave i really feel like my only purpose ...
3,__label__suicidal,i did something on the 2 of october i overdose...
4,__label__suicidal,i feel like no one cares i just want to die ma...
...,...,...
7117,__label__nonsuicidal,rented marley and me if you watch it be prepar...
7118,__label__nonsuicidal,another night all alone and it s really spooky
7119,__label__nonsuicidal,you okay laura
7120,__label__nonsuicidal,goodnight twitterlandtime for me to go to drea...


In [ ]:
d['label'].value_counts()
n=d[d.label=='__label__nonsuicidal']
s=d[d.label=='__label__suicidal']
s

,label,text
0,__label__suicidal,my life is meaningless i just want to end my l...
1,__label__suicidal,muttering i wanna die to myself daily for a fe...
2,__label__suicidal,work slave i really feel like my only purpose ...
3,__label__suicidal,i did something on the 2 of october i overdose...
4,__label__suicidal,i feel like no one cares i just want to die ma...
...,...,...
6881,__label__suicidal,my biggest regret i have felt more alone than ...
6882,__label__suicidal,cheated on my gf i want to kill myself i m28 c...
6883,__label__suicidal,i m unwanted and i don t want to live in a wor...
6884,__label__suicidal,i just want everyone i love to know it isn t t...


**Data Preparation**

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Data Preprocessing**

In [ ]:
arr=[]
def data_preprocess(t):

  #special characters
  t = re.sub('[^a-zA-Z]', ' ', t)
  #lower case
  t = t.lower()
  t = t.split()
  #stemming and stopwords removal
  t = [ps.stem(word) for word in t if not word in set(all_stopwords)]
  #join 
  t = ' '.join(t)
  #arr.append(t)
  token=nltk.word_tokenize(t)
  arr.append(token)
  return t;
 
data_preprocess('I am not happy!')

'not happi'

In [ ]:
#arrr=[]
d['clean_text']=d['text'].apply(lambda x:data_preprocess(x))





Preprocessed data

In [ ]:
d.head()

,label,text,clean_text
0,__label__suicidal,my life is meaningless i just want to end my l...,life meaningless want end life badli life comp...
1,__label__suicidal,muttering i wanna die to myself daily for a fe...,mutter wanna die daili month feel worthless sh...
2,__label__suicidal,work slave i really feel like my only purpose ...,work slave realli feel like purpos life make h...
3,__label__suicidal,i did something on the 2 of october i overdose...,someth octob overdos felt alon horribl hospit ...
4,__label__suicidal,i feel like no one cares i just want to die ma...,feel like one care want die mayb feel less lone


Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(max_features=5000)


In [ ]:
x = cv.fit_transform(d['clean_text'])

#df=pd.DataFrame(x.toarray(), columns=cv.get_feature_names())
#df.head()
x

<7122x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 230846 stored elements in Compressed Sparse Row format>

In [ ]:
#import pickle
#bow='/content/drive/MyDrive/ehealthdatset/bow.pkl'
#pickle.dump(cv, open(bow, "wb"))


**Training and Testing**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
x_train,x_test,y_train,y_test=train_test_split(x,d['label'],test_size=0.20,random_state=0)


x_test

<1425x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 46084 stored elements in Compressed Sparse Row format>

**Evaluation**

Logistic Regression

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
lr=LogisticRegression(class_weight="weighted")
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)

print(accuracy_score(y_test,y_pred))

0.9663157894736842


**Prediction**

In [ ]:
x='I feel like my life is never going to get better!!'
x=data_preprocess(x)
print(x)
v=cv.transform([x])

lr.predict(v)

feel like life never go get better


array(['__label__suicidal'], dtype=object)

In [ ]:
x='Nowadays i feel more energtic and i look forward for a wonderful day.'
x=data_preprocess(x)
print(x)
v=cv.transform([x])

lr.predict(v)

nowaday feel energt look forward wonder day


array(['__label__nonsuicidal'], dtype=object)

Saving the model

In [ ]:
import pickle
bow='/content/drive/MyDrive/ehealthdatset/model3.pkl'
pickle.dump(cv, open(bow, "wb"))
